# Asteroid Classification
This notebook contains a jupyter notebook to train and test a dense neural network for classifying asteroids as hazardous or not. Because of data imbalance, in testing the network only achieved about 75% binary accuracy, however a much greater percentage of these were false positive than negatives.

In [1]:
import tensorflow as tf 
from tensorflow import keras 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import io
import os
import tempfile

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

relative_velocity_flag = False
miss_distance_flag = False

### Data Preprocessing

In [2]:
csv_link = 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/Final-Project/ExtraKGs/celesital-bodies/neo_v2.csv'
csv_file = requests.get(csv_link).content
dataset = pd.read_csv(io.StringIO(csv_file.decode('utf-8')))

In [3]:
change_to_float = {column:float for column, dtype in zip(dataset.columns.tolist(), dataset.dtypes.tolist()) if dtype == np.bool_}
dataset = dataset.astype(change_to_float)
dataset.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,Earth,0.0,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,Earth,0.0,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,Earth,0.0,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,Earth,0.0,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,Earth,0.0,20.09,1.0


In [4]:
neg, pos = np.bincount(dataset['hazardous'])
total = neg + pos
print(f"Percent positive is {(pos / neg) * 100} %")

Percent positive is 10.781013707985755 %


In [5]:
try:
    dataset_processed = dataset.drop(['sentry_object','orbiting_body'], axis=1)
except (AttributeError, KeyError):
    print("already removed")

In [6]:
dataset_processed.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,2162635,162635 (2000 SS164),1.198271,2.679415,13569.249224,5.483974e+07,16.73,0.0
1,2277475,277475 (2005 WK4),0.265800,0.594347,73588.726663,6.143813e+07,20.00,1.0
2,2512244,512244 (2015 YE18),0.722030,1.614507,114258.692129,4.979872e+07,17.83,0.0
3,3596030,(2012 BV13),0.096506,0.215794,24764.303138,2.543497e+07,22.20,0.0
4,3667127,(2014 GE35),0.255009,0.570217,42737.733765,4.627557e+07,20.09,1.0


In [7]:
def scale_down(dataset=None, column=None):
    initial_range = np.amax(dataset[column]) - np.amin(dataset[column])
    eps = 0.001
    dataset[column] = np.log(dataset[column] + eps)
    end_range = np.amax(dataset[column]) - np.amin(dataset[column])

    return initial_range, end_range

In [8]:
if not relative_velocity_flag:
    initial_range, end_range = scale_down(column='miss_distance', dataset=dataset_processed)
    relative_velocity_flag = True
else:
    print("scaling already complete")

if not miss_distance_flag:
    initial_range, end_range = scale_down(column='relative_velocity', dataset=dataset_processed)
    miss_distance_flag = True
else:
    print("scaling already complete")

In [9]:
scaler = StandardScaler()
train_data, test_data = train_test_split(dataset_processed.drop(['id', 'name'], axis=1), test_size=0.1)
_, regurgitate_data = train_test_split(dataset, test_size=0.1)
train_data, validation_data = train_test_split(train_data, test_size=0.1)

train_labels, test_labels, validation_labels = (np.array(train_data.pop('hazardous')),
                                                np.array(test_data.pop('hazardous')), 
                                                np.array(validation_data.pop('hazardous')))
bool_train_labels = train_labels != 0
train_features, test_features, validation_features = (np.clip(scaler.fit_transform(np.array(train_data)), 0, 60),
                                                      np.clip(scaler.fit_transform(np.array(test_data)), 0, 60), 
                                                      np.clip(scaler.fit_transform(np.array(validation_data)), 0, 60))

### Model Creation

In [10]:
model_metrics = [keras.metrics.BinaryAccuracy(),
                       keras.metrics.Precision(),
                       keras.metrics.Recall(),
                       keras.metrics.TruePositives(),
                       keras.metrics.TrueNegatives(),
                       keras.metrics.FalseNegatives(),
                       keras.metrics.FalsePositives(),
                       keras.metrics.AUC(name='auc'),
                       keras.metrics.AUC(name='prc', curve='PR')
                       ]

In [11]:
def make_model(metrics=model_metrics, output_bias=None):
    if output_bias:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = tf.keras.Sequential([
        keras.layers.Dense(5, activation='relu', input_shape=(train_features.shape[-1],), 
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(5, activation='linear', activity_regularizer=keras.regularizers.L1(0.0002)),
        keras.layers.Dense(5, activation='relu',
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(3, activation='linear', activity_regularizer=keras.regularizers.L1(0.0002)),
        keras.layers.Dense(3, activation='relu',
                            kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias)
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )
    
    return model

In [12]:
# epochs, batch size, and early stopping
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True
)

# Setting the initial bias
initial_bias = np.log([pos/neg])
model = make_model(output_bias=initial_bias)
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [13]:
# classification weights
weight_for_0 = (1 / neg) * (total / 2)
weight_for_1 = (1 / pos) * (total / 2)

class_weights = {0: weight_for_0, 1: weight_for_1}

print(f'Non-Hazardous Weight: {weight_for_0}\nHazardous Weight: {weight_for_1}')

Non-Hazardous Weight: 0.5539050685399288
Hazardous Weight: 5.137782805429865


In [14]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)
weighted_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 5)                 30        
                                                                 
 dropout_3 (Dropout)         (None, 5)                 0         
                                                                 
 dense_7 (Dense)             (None, 5)                 30        
                                                                 
 dense_8 (Dense)             (None, 5)                 30        
                                                                 
 dropout_4 (Dropout)         (None, 5)                 0         
                                                                 
 dense_9 (Dense)             (None, 3)                 18        
                                                                 
 dense_10 (Dense)            (None, 3)                

In [15]:
weighted_model.fit(train_features, 
                    train_labels, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=[early_stopping], 
                    validation_data=(validation_features, validation_labels), 
                    class_weight=class_weights)

Epoch 1/100
36/36 [==============================] - 12s 112ms/step - loss: 1.3502 - binary_accuracy: 0.9036 - precision: 0.0000e+00 - recall: 0.0000e+00 - true_positives: 0.0000e+00 - true_negatives: 66483.0000 - false_negatives: 7093.0000 - false_positives: 0.0000e+00 - auc: 0.4235 - prc: 0.0907 - val_loss: 0.3438 - val_binary_accuracy: 0.8975 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 - val_true_negatives: 7338.0000 - val_false_negatives: 838.0000 - val_false_positives: 0.0000e+00 - val_auc: 0.4288 - val_prc: 0.0950
Epoch 2/100
36/36 [==============================] - 0s 13ms/step - loss: 1.1065 - binary_accuracy: 0.9036 - precision: 0.0000e+00 - recall: 0.0000e+00 - true_positives: 0.0000e+00 - true_negatives: 66483.0000 - false_negatives: 7093.0000 - false_positives: 0.0000e+00 - auc: 0.4961 - prc: 0.1112 - val_loss: 0.3439 - val_binary_accuracy: 0.8975 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_positives: 0.0000e+00 

In [16]:
weighted_model.evaluate(test_features, test_labels, batch_size=BATCH_SIZE, verbose=1)

5/5 [==============================] - 0s 6ms/step - loss: 0.3597 - binary_accuracy: 0.7686 - precision: 0.2843 - recall: 0.8647 - true_positives: 786.0000 - true_negatives: 6196.0000 - false_negatives: 123.0000 - false_positives: 1979.0000 - auc: 0.8746 - prc: 0.3512


[0.35969623923301697,
 0.7686041593551636,
 0.2842676341533661,
 0.8646864891052246,
 786.0,
 6196.0,
 123.0,
 1979.0,
 0.8746435642242432,
 0.3512299358844757]

In [17]:
results = weighted_model.predict(test_features)
results[results<0.5]=0
results[results>0.5]=1

284/284 [==============================] - 2s 4ms/step


In [18]:
len(results)

9084

In [19]:
regurgitate_data['predicted_values'] = results

In [22]:
regurgitate_data.head()

,id,name,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,orbiting_body,sentry_object,absolute_magnitude,hazardous,predicted_values
7403,3678630,(2014 OL339),0.069913,0.156329,33265.239717,3.943174e+07,Earth,0.0,22.90,0.0,0.0
68747,2369057,369057 (2008 DK5),0.146068,0.326618,85407.623874,2.765249e+07,Earth,0.0,21.30,1.0,1.0
67329,3601615,(2012 FW23),0.033462,0.074824,23596.338308,2.394144e+07,Earth,0.0,24.50,0.0,0.0
35401,54192215,(2021 RF2),0.018817,0.042077,26849.848210,1.461445e+07,Earth,0.0,25.75,0.0,0.0
42280,3633054,(2013 FD8),0.019256,0.043057,33739.973279,3.246799e+06,Earth,0.0,25.70,0.0,1.0


In [ ]:
regurgitate_data.to_csv(path_or_buf=r'C:\Users\ahuma\Desktop\python projects\REHS2022\Final-Project\ExtraKGs\celesital-bodies\processed_data.csv')

In [21]:
#keras.backend.clear_session()